In [1]:
from marley.utils.ipstartup import *
from marley import tif, config as c
import rasterio
from rasterstats import zonal_stats
import geopandas as gpd
os.chdir(c.localpath)

# other cropped areas

In [4]:
lightsfile = r"grid3\NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG_median_avg_rad.tif"
d("**** with lights **********")
d(zonal_stats("vectors/examples/ajaokuta.gpkg", lightsfile))

d("**** without lights **********")
d(zonal_stats("vectors/examples/omuo.gpkg", lightsfile))
d(zonal_stats("vectors/examples/agbaja.gpkg", lightsfile))
d(zonal_stats("vectors/examples/agbede all.kml", lightsfile))

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:29)


'**** with lights **********'

[{'min': 0.350009948015213,
  'max': 3.699918746948242,
  'mean': 1.6705363412921348,
  'count': 2047}]

'**** without lights **********'

[{'min': 0.12100640684366226,
  'max': 0.3949834406375885,
  'mean': 0.21725114297368386,
  'count': 574}]

[{'min': 0.2040025144815445,
  'max': 0.5649789571762085,
  'mean': 0.32037184926008017,
  'count': 267}]

[{'min': 0.19199931621551514,
  'max': 0.2849973738193512,
  'mean': 0.24325728684328915,
  'count': 89}]

time: 535 ms


In [7]:
popfile = r"grid3/GRID3_NGA_PopEst_v1_1_mean_float.tif"
data = rasterio.open(popfile).read()[0]
data[data<0] = 0
data[np.isnan(data)] = 0
d(zonal_stats("vectors/examples/agbede all.kml", data, affine=rasterio.open(popfile).transform, stats=["sum"]))
zonal_stats("vectors/examples/richifa.kml", data, affine=rasterio.open(popfile).transform, stats=["sum"])

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:30)


[{'sum': 396.9110016822815}]

[{'sum': 12652.824938297272}]

time: 2.51 s


In [8]:
popfile = r"dmsp/GRID3_NGA_PopEst_v1_1_mean_float.tif"
data = rasterio.open(popfile).read()[0]
d(zonal_stats("vectors/examples/agbede all.kml", data, affine=rasterio.open(popfile).transform, stats=["sum"]))
zonal_stats("vectors/examples/richifa.kml", data, affine=rasterio.open(popfile).transform, stats=["sum"])

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:30)


[{'sum': 1274.3035315627521}]

[{'sum': 14779.40737784428}]

time: 631 ms


# village agbede - apply KML vector to raster

* cut the shape files in google maps pro for 4 areas around agbede; one super-area around the whole settlement; and one round remote village south west
* rasterstats to get data for those areas from the population raster
* manually estimated number of houses
* calculated area km2 and density pop/km2

check:
* scale is 1 pixel = 100m
* agbede north is about 200m*50m = .2 * .05=.01km2 => looks OK

results:
* at high resolution it is not accurate e.g. the forest has more people than the village
* that is only 2 pixels for agbede north with 30 houses
* as you zoom out then you get 163 people in 100 houses. still a bit low.
* looks like it could be improved by increasing the weight given to settlements

In [9]:
# compare count of houses to worldpop
shapes = ["vectors/examples/agbede north.kml",\
          "vectors/examples/agbede north east.kml",\
          "vectors/examples/agbede south.kml",\
          "vectors/examples/agbede south east.kml",
          "vectors/examples/agbede all.kml",
          "vectors/examples/unnamed swest agbedde.kml"]

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:31)


time: 0 ns


In [10]:
def getstats(shapes, tifname):
    # get as array to allow replacement of -3.4e-34
    data = rasterio.open(tifname).read()[0]
    data[data<0] = 0
    data[np.isnan(data)] = 0
    d(f"total population={data.sum()}")
    
    # create df of shapes
    allstats = []
    for shape in shapes:
        row = zonal_stats(shape, data, affine=rasterio.open(tifname).meta["transform"])
        # list of 1 polygon
        row[0].update(geometry = gpd.read_file(shape).geometry.values)
        allstats.extend(row)
    df = pd.DataFrame.from_records(allstats)
    
    # format df
    df["filename"] = shapes
    df["sum"] = df["mean"]*df["count"]
    df["houses"] = [30,0,0,70, 100, 70]
    df["area"] = df.geometry.apply(lambda x: x.area.sum())*10000
    df["density"] = df["sum"]/df.area
    return df.round(3).drop(columns=["geometry"])

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:31)


time: 10.2 ms


In [11]:
getstats(shapes, "wpop/WorldPop_GP_100m_pop_median_population.tif")

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:31)


'total population=209758128.0'

,count,max,mean,min,filename,sum,houses,area,density
0,2,1.837,1.829,1.821,vectors/examples/agbede north.kml,3.658,30,0.011,326.203
1,10,2.085,2.033,1.896,vectors/examples/agbede north east.kml,20.326,0,0.058,350.756
2,2,1.845,1.817,1.789,vectors/examples/agbede south.kml,3.634,0,0.017,208.548
3,7,2.051,2.035,2.010,vectors/examples/agbede south east.kml,14.242,70,0.047,306.016
4,89,2.085,1.839,1.335,vectors/examples/agbede all.kml,163.691,100,0.614,266.697
5,8,0.259,0.254,0.246,vectors/examples/unnamed swest agbedde.kml,2.030,70,0.062,32.831


time: 8.53 s


In [12]:
getstats(shapes, r"grid3/GRID3_NGA_PopEst_v1_1_mean_float.tif")

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-14 18:31)


'total population=179876176.0'

,count,max,mean,min,filename,sum,houses,area,density
0,2,35.124,24.659,14.194,vectors/examples/agbede north.kml,49.318,30,0.011,4398.301
1,10,11.175,1.433,0.000,vectors/examples/agbede north east.kml,14.332,0,0.058,247.315
2,2,23.006,16.586,10.165,vectors/examples/agbede south.kml,33.171,0,0.017,1903.559
3,7,54.212,26.604,2.648,vectors/examples/agbede south east.kml,186.226,70,0.047,4001.343
4,89,54.212,4.460,0.000,vectors/examples/agbede all.kml,396.911,100,0.614,646.678
5,8,61.856,21.025,0.000,vectors/examples/unnamed swest agbedde.kml,168.204,70,0.062,2720.216


time: 2.43 s
